In [1]:
import pandas as pd
import numpy as np
import itertools


In [2]:
#import csv file
try:
    df_train = pd.read_csv('../data/train.csv')
    df_test = pd.read_csv('../data/test.csv')
except FileNotFoundError:
    print("Error: files not found")
    exit()

df_train.info()
df_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Id                    8000 non-null   int64 
 1   Therapy Hours         8000 non-null   int64 
 2   Initial Health Score  8000 non-null   int64 
 3   Lifestyle Activities  8000 non-null   object
 4   Average Sleep Hours   8000 non-null   int64 
 5   Follow-Up Sessions    8000 non-null   int64 
 6   Recovery Index        8000 non-null   int64 
dtypes: int64(6), object(1)
memory usage: 437.6+ KB


,Id,Therapy Hours,Initial Health Score,Lifestyle Activities,Average Sleep Hours,Follow-Up Sessions,Recovery Index
0,9255,5,49,No,7,5,36
1,1562,2,48,Yes,7,6,25
2,1671,2,81,No,7,2,59
3,6088,2,46,No,6,1,22
4,6670,8,47,No,9,0,40


In [3]:
test_ids = df_test['Id']

df_train_id_dropped = df_train.drop('Id', axis=1)
df_test_id_dropped = df_test.drop('Id', axis=1)

for df in [df_train_id_dropped, df_test_id_dropped]:
    df['Lifestyle Activities'] = df['Lifestyle Activities'].apply(lambda x: 1 if x == 'Yes' else 0)

df_train_id_dropped.info()
df_train_id_dropped.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   Therapy Hours         8000 non-null   int64
 1   Initial Health Score  8000 non-null   int64
 2   Lifestyle Activities  8000 non-null   int64
 3   Average Sleep Hours   8000 non-null   int64
 4   Follow-Up Sessions    8000 non-null   int64
 5   Recovery Index        8000 non-null   int64
dtypes: int64(6)
memory usage: 375.1 KB


,Therapy Hours,Initial Health Score,Lifestyle Activities,Average Sleep Hours,Follow-Up Sessions,Recovery Index
0,5,49,0,7,5,36
1,2,48,1,7,6,25
2,2,81,0,7,2,59
3,2,46,0,6,1,22
4,8,47,0,9,0,40


In [4]:
train_output_path = '../data/train_processed.csv'
df_train_id_dropped.to_csv(train_output_path, index=False)
print(f"Processed training data saved to: {train_output_path}")

test_output_path = '../data/test_processed.csv'
df_test_id_dropped.to_csv(test_output_path, index=False)
print(f"Processed test data saved to: {test_output_path}")

ids_output_path = '../data/test_ids.csv'
test_ids.to_csv(ids_output_path, index=False)
print(f"Test IDs saved to: {ids_output_path}")

Processed training data saved to: ../data/train_processed.csv
Processed test data saved to: ../data/test_processed.csv
Test IDs saved to: ../data/test_ids.csv


In [5]:
# --- Feature Engineering: Add 5C2 Interaction Features ---

print("Original data shape:", df_train_id_dropped.shape)

# Original list of 5 features
features = ['Therapy Hours', 'Initial Health Score', 'Lifestyle Activities', 'Average Sleep Hours', 'Follow-Up Sessions']

# Get all 2-feature combinations (10 pairs)
all_combinations = list(itertools.combinations(features, 2))
print(f"Generating {len(all_combinations)} new interaction features...")

for df in [df_train_id_dropped, df_test_id_dropped]:
    for col1, col2 in all_combinations:
        new_col_name = f"{col1}_x_{col2}".replace(' ', '_')
        df[new_col_name] = df[col1] * df[col2]

print("All 10 new interaction features added.")
print("New data shape:", df_train_id_dropped.shape)
print("\n--- Training Data with New Interaction Features ---")
display(df_train_id_dropped.head())

Original data shape: (8000, 6)
Generating 10 new interaction features...
All 10 new interaction features added.
New data shape: (8000, 16)

--- Training Data with New Interaction Features ---


,Therapy Hours,Initial Health Score,Lifestyle Activities,Average Sleep Hours,Follow-Up Sessions,Recovery Index,Therapy_Hours_x_Initial_Health_Score,Therapy_Hours_x_Lifestyle_Activities,Therapy_Hours_x_Average_Sleep_Hours,Therapy_Hours_x_Follow-Up_Sessions,Initial_Health_Score_x_Lifestyle_Activities,Initial_Health_Score_x_Average_Sleep_Hours,Initial_Health_Score_x_Follow-Up_Sessions,Lifestyle_Activities_x_Average_Sleep_Hours,Lifestyle_Activities_x_Follow-Up_Sessions,Average_Sleep_Hours_x_Follow-Up_Sessions
0,5,49,0,7,5,36,245,0,35,25,0,343,245,0,0,35
1,2,48,1,7,6,25,96,2,14,12,48,336,288,7,6,42
2,2,81,0,7,2,59,162,0,14,4,0,567,162,0,0,14
3,2,46,0,6,1,22,92,0,12,2,0,276,46,0,0,6
4,8,47,0,9,0,40,376,0,72,0,0,423,0,0,0,0


In [6]:
train_output_path = '../data/train_processed_v2.csv'
df_train_id_dropped.to_csv(train_output_path, index=False)
print(f"Processed training data saved to: {train_output_path}")

test_output_path = '../data/test_processed_v2.csv'
df_test_id_dropped.to_csv(test_output_path, index=False)
print(f"Processed test data saved to: {test_output_path}")

ids_output_path = '../data/test_ids.csv'
test_ids.to_csv(ids_output_path, index=False)
print(f"Test IDs saved to: {ids_output_path}")

Processed training data saved to: ../data/train_processed_v2.csv
Processed test data saved to: ../data/test_processed_v2.csv
Test IDs saved to: ../data/test_ids.csv
